In [5]:
import torch
import time
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader


class ResNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResNetBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

 
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += self.shortcut(residual)
        out = self.relu(out)
        return out

# Define ResNet-10
class ResNet10(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet10, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.blocks = self._make_blocks()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(64, num_classes)

    def _make_blocks(self):
        layers = []
        for _ in range(10):
            layers.append(ResNetBlock(64, 64))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.blocks(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

# Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset =  CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader =  DataLoader(test_dataset, batch_size=64, shuffle=True)


model = ResNet10().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Training loop
for epoch in range(300):
    model.train()
    start_time = time.time()
    total_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    end_time = time.time()
    epoch_time = end_time - start_time
    average_loss = total_loss / len(train_loader)

    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}/{300}, Loss: {average_loss:.4f}, Epoch Time: {epoch_time:.2f} seconds')

# Evaluate the model
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

valid_accuracy = correct / total
print(f'Test Accuracy: {valid_accuracy * 100:.2f}%')

Files already downloaded and verified
Files already downloaded and verified
Epoch 1/300, Loss: 1.4812, Epoch Time: 29.80 seconds
Epoch 11/300, Loss: 0.3457, Epoch Time: 47.17 seconds
Epoch 21/300, Loss: 0.1442, Epoch Time: 47.12 seconds
Epoch 31/300, Loss: 0.0570, Epoch Time: 45.75 seconds
Epoch 41/300, Loss: 0.0242, Epoch Time: 45.92 seconds
Epoch 51/300, Loss: 0.0214, Epoch Time: 47.69 seconds
Epoch 61/300, Loss: 0.0045, Epoch Time: 46.39 seconds
Epoch 71/300, Loss: 0.0006, Epoch Time: 44.95 seconds
Epoch 81/300, Loss: 0.0009, Epoch Time: 44.64 seconds
Epoch 91/300, Loss: 0.0025, Epoch Time: 46.18 seconds
Epoch 101/300, Loss: 0.0067, Epoch Time: 45.10 seconds
Epoch 111/300, Loss: 0.0011, Epoch Time: 47.27 seconds
Epoch 121/300, Loss: 0.0003, Epoch Time: 45.63 seconds
Epoch 131/300, Loss: 0.0057, Epoch Time: 47.02 seconds
Epoch 141/300, Loss: 0.0004, Epoch Time: 45.56 seconds
Epoch 151/300, Loss: 0.0005, Epoch Time: 45.68 seconds
Epoch 161/300, Loss: 0.0002, Epoch Time: 44.68 seconds
